In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.util import ngrams
from nltk.corpus import stopwords
from wordcloud import WordCloud


%matplotlib inline

plt.style.use('default')

sns.set(style='whitegrid')

In [ ]:
df = pd.read_csv('data/events.csv', low_memory=False)

# Análisis de términos de búsqueda

Primero vamos a revisar los términos de búsqueda más usados, y ver cuál es el panorama.

In [ ]:
terminos_busqueda = df.loc[df['search_term'].notnull(), ['timestamp', 'person', 'event', 'model', 'search_term', 'skus']]
terminos_busqueda['timestamp'] = pd.to_datetime(terminos_busqueda['timestamp']).dt.date
terminos_busqueda['month'] = pd.to_datetime(terminos_busqueda['timestamp']).dt.month
terminos_busqueda['search_term'] = terminos_busqueda['search_term'].apply(lambda x: x.lower())
terminos_busqueda['search_term'] = terminos_busqueda['search_term'].str.replace('sansung', 'samsung')

## Word Cloud de los términos de búsqueda presentes en el dataset

In [ ]:
wordcloud = WordCloud(background_color='black',
                      max_words=200
                     ).generate(str(terminos_busqueda['search_term']))
plt.figure(figsize=[20,4])
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

## Top 10 búsquedas de usuarios

In [ ]:
barplot_st = terminos_busqueda['search_term'].value_counts().head(10)
plt.figure(figsize=[12,6])
g = sns.barplot(x=barplot_st.index, y=barplot_st.values, palette='hls')
g.set_title('Top 10 búsquedas de usuarios', fontsize=20)
g.set_xlabel('Búsqueda', fontsize=18)
g.set_ylabel('Frecuencia', fontsize=18)
plt.xticks(rotation=45)
plt.show()

# Otro enfoque para analizar los términos de búsqueda

Si bien los términos de búsqueda son un punto de interés para nuestro análisis, creemos que se puede profundizar más sobre el mismo. En particular, podríamos analizar la frecuencia de n-gramas en estas búsquedas, para agrupar mejor las mismas.

### ¿Cuál es la motivación de este enfoque? 

Puesto que estamos analizando el input consumido por el search engine del sitio de Trocafone, tal vez podamos encontrar algo interesante, como nuevas keywords para los productos ofrecidos en el sitio.

### ¿Cómo vamos a realizar este análisis?

En primer lugar, debemos filtrar las _stopwords_ del lenguaje que presenta mayoría (en este caso, portugués). El fin de esto es prevenir que aparezcan en nuestros reportes palabras de uso común pero que carecen de contenido, como por ejemplo `por, de, mais`.

Luego utilizaremos la biblioteca `nltk` para poder procesar cada término de búsqueda y extraer los n-gramas que nos interesen.

Finalmente presentaremos la información por medio de gráficos de barra, ya que nos interesa ver los n-gramas que presenten mayor frecuencia.

In [ ]:
terminos_sin_filtrar = terminos_busqueda['search_term'].tolist()

## Vamos a cachear las stopwords ya que queremos evitar un cuello de botella al cargarlo constantemente.
cached_stopwords = stopwords.words('portuguese')

terminos_filtrados = []

for frase in terminos_sin_filtrar:
    terminos_filtrados.append(' '.join([palabra for palabra in frase.split(' ') if palabra not in cached_stopwords]))

In [ ]:
def obtener_ngramas(frases, n):
    data = []
    for frase in frases:
        tokens = [token for token in frase.split(" ") if token != ""]
        ngramas = list(ngrams(tokens, n))
        for _ in ngramas:
            data.append(_)
    res = [(lambda x: ' '.join(x))(x) for x in data]
    return res

## Unigramas

In [ ]:
unigrams = obtener_ngramas(terminos_filtrados, 1)
serie = pd.Series(x for x in unigrams).value_counts().head(10)
plt.figure(figsize=[12,8])
g = sns.barplot(x=serie.index, y=serie.values, palette='hls')
g.set_title('Frecuencia de unigramas en términos de búsqueda', fontsize=18)
g.set_xlabel('Unigramas', fontsize=16)
g.set_ylabel('Frecuencia', fontsize=16)
plt.xticks(rotation=45)
plt.show()

## 2-gramas

In [ ]:
bigrams = obtener_ngramas(terminos_filtrados, 2)
serie = pd.Series(x for x in bigrams).value_counts().head(10)
plt.figure(figsize=[12,8])
g = sns.barplot(x=serie.index, y=serie.values, palette='hls')
g.set_title('Frecuencia de 2-gramas en términos de búsqueda', fontsize=18)
g.set_xlabel('2-gramas', fontsize=16)
g.set_ylabel('Frecuencia', fontsize=16)
plt.xticks(rotation=45)
plt.show()

## 3-gramas

In [ ]:
trigrams = obtener_ngramas(terminos_filtrados, 3)
serie = pd.Series(x for x in trigrams).value_counts().head(10)
plt.figure(figsize=[12,8])
g = sns.barplot(x=serie.index, y=serie.values, palette='hls')
g.set_title('Frecuencia de 3-gramas en términos de búsqueda', fontsize=18)
g.set_xlabel('3-gramas', fontsize=16)
g.set_ylabel('Frecuencia', fontsize=16)
plt.xticks(rotation=45)
plt.show()

## 4-gramas

In [ ]:
fourgrams = obtener_ngramas(terminos_filtrados, 4)
serie = pd.Series(x for x in fourgrams).value_counts().head(10)
plt.figure(figsize=[12,6])
g = sns.barplot(x=serie.index, y=serie.values, palette='hls')
g.set_title('Frecuencia de 4-gramas en términos de búsqueda', fontsize=18)
g.set_xlabel('4-gramas', fontsize=16)
g.set_ylabel('Frecuencia', fontsize=16)
plt.xticks(rotation=90)
plt.show()